In [32]:
import spacy
import csv
import pandas as pd
import math
from spacy.tokens import DocBin
from spacy import displacy

In [33]:
def find_largest_common_substring(str1, str2):
    m = len(str1)
    n = len(str2)

    # Create a table to store the lengths of the common substrings
    table = [[0] * (n + 1) for _ in range(m + 1)]

    # Variables to store the length and ending position of the longest common substring
    max_length = 0
    end_position = 0

    # Iterate through each character of str1 and str2
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                table[i][j] = table[i - 1][j - 1] + 1
                if table[i][j] > max_length:
                    max_length = table[i][j]
                    end_position = i

    # Extract the longest common substring
    longest_common_substring = str1[end_position - max_length: end_position]

    return longest_common_substring.strip()

In [34]:
df = pd.read_csv('2complete.csv')
# specs_df = pd.read_csv('table_data_refrigerator.csv')
df = df.sample(frac = 1)
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)
# test_df
df.head()

,ID,Title,Brand,Model
2035,2036,"Samsung S1030 comes with a 1/1.8"" (~ 7.11 x 5...",Samsung,S1030
2045,2046,"Samsung L77 comes with a 1/2.5"" (~ 5.75 x 4.3...",Samsung,L77
2785,2786,"Casio QV-2100 comes with a 1/2.7"" (~ 5.33 x 4...",Casio,QV-2100
261,262,"Canon S60 comes with a 1/1.8"" (~ 7.11 x 5.33...",Canon,PowerShot S60
373,374,"Fujifilm S8200 comes with a 1/2.3"" (~ 6.16 x...",Fujifilm,FinePix S8200


In [35]:
import random

def jumble_sentence(sentence):
    first_space=sentence.find(' ')

    before_comes=sentence.find('comes')

    words=[]
    words.append(sentence[:first_space])
    words.append(sentence[first_space+1:before_comes])

    words.extend(sentence[before_comes:].split())
    random.shuffle(words)
    jumbled_sentence = ' '.join(words)
    return jumbled_sentence

In [36]:
data=[]
for ind in df.index:
    id = df['ID'][ind]
    title = df['Title'][ind]
    jumb_title = jumble_sentence(title)
    brand = df['Brand'][ind]
    model = df['Model'][ind]
    brand_start = jumb_title.find(brand)
    x = find_largest_common_substring(model,jumb_title)
    y= jumb_title.find(x)
    data.append({'ID':id, 'Title': jumb_title, 'Brand': brand, 'Model': model, 'brand_start': brand_start, 'brand_end': brand_start + len(brand), 'model_start': y, 'model_end': y + len(x)})
train = data[:math.floor(len(data)*0.6)]
test = data[math.floor(len(data)*0.6):]

In [37]:
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)

In [38]:
training_data = {'classes':['BRAND', 'MODEL'], 'annotations':[]}
for ex in train:
    temp_dict = {}
    temp_dict['text'] = ex['Title']
    temp_dict['entities'] = []
    temp_dict['entities'].append((ex['brand_start'], ex['brand_end'], 'BRAND'))
    temp_dict['entities'].append((ex['model_start'], ex['model_end'], 'MODEL'))
    training_data['annotations'].append(temp_dict)
training_data['annotations'][0]

{'text': '37.90 S1030  diagonal has Samsung of comes a mm (0.35") and area which 5.33 a mm². mm) a x surface of CCD (~ sensor, 7.11 1/1.8" with 8.89',
 'entities': [(26, 33, 'BRAND'), (6, 11, 'MODEL')]}

In [39]:
nlp = spacy.blank('en')
doc_bin = DocBin()

for training_example  in (training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [49]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner('R7 from Fujifilm comes with a ')
for ent in doc.ents:
    print(ent.text, ent.label_)
colors = {"MODEL": "#F67DE3", "BRAND": "#7DF6D9"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Fujifilm BRAND
